In [1]:
import torch
import torchaudio
import os

In [3]:
audio_file = '/mnt/user/saksham/AV_robust/AV-C-Robustness-Benchmark/data_recipe/src/noise_files/all_sounds/concert2.mp3'
output_folder = '/mnt/user/saksham/AV_robust/AV-C-Robustness-Benchmark/data_recipe/src/noise_files/concert'

def split_and_save_audio(audio_file, output_folder, chunk_duration=10, target_sr=16000):
    waveform, sr = torchaudio.load(audio_file)  # shape: [channels, samples]
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)
    if sr != target_sr:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sr)
        waveform = resampler(waveform)
        sr = target_sr  # Update sample rate
    
    chunk_samples = chunk_duration * sr
    total_samples = waveform.shape[1]

    for i, start in enumerate(range(0, total_samples, chunk_samples)):
        end = min(start + chunk_samples, total_samples)
        chunk = waveform[:, start:end]

        if chunk.shape[1] < chunk_samples:
            continue
            pad_size = chunk_samples - chunk.shape[1]
            chunk = torch.nn.functional.pad(chunk, (0, pad_size))

        chunk_path = os.path.join(output_folder, f"concert2_{i+1}.wav")
        torchaudio.save(chunk_path, chunk, sample_rate=sr)

split_and_save_audio(audio_file, output_folder, chunk_duration=10)
